In [539]:
import pandas as pd
import requests
from faker import Faker
import re
import phonenumbers
import numpy as np
from datetime import datetime, timedelta

import sqlite3

In [1]:
!pip install phonenumbers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 17.0 MB/s eta 0:00:00m eta 0:00:010:00:01


In [503]:
#Extract data from API

response= requests.get('https://jsonplaceholder.typicode.com/users')
users = response.json()

In [504]:
users_df = pd.DataFrame(users)
users_df_copy = users_df.copy()
users_df

,id,name,username,email,address,phone,website,company
0,1,Leanne Graham,Bret,Sincere@april.biz,"{'street': 'Kulas Light', 'suite': 'Apt. 556',...",1-770-736-8031 x56442,hildegard.org,"{'name': 'Romaguera-Crona', 'catchPhrase': 'Mu..."
1,2,Ervin Howell,Antonette,Shanna@melissa.tv,"{'street': 'Victor Plains', 'suite': 'Suite 87...",010-692-6593 x09125,anastasia.net,"{'name': 'Deckow-Crist', 'catchPhrase': 'Proac..."
2,3,Clementine Bauch,Samantha,Nathan@yesenia.net,"{'street': 'Douglas Extension', 'suite': 'Suit...",1-463-123-4447,ramiro.info,"{'name': 'Romaguera-Jacobson', 'catchPhrase': ..."
3,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,"{'street': 'Hoeger Mall', 'suite': 'Apt. 692',...",493-170-9623 x156,kale.biz,"{'name': 'Robel-Corkery', 'catchPhrase': 'Mult..."
4,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,"{'street': 'Skiles Walks', 'suite': 'Suite 351...",(254)954-1289,demarco.info,"{'name': 'Keebler LLC', 'catchPhrase': 'User-c..."
5,6,Mrs. Dennis Schulist,Leopoldo_Corkery,Karley_Dach@jasper.info,"{'street': 'Norberto Crossing', 'suite': 'Apt....",1-477-935-8478 x6430,ola.org,"{'name': 'Considine-Lockman', 'catchPhrase': '..."
6,7,Kurtis Weissnat,Elwyn.Skiles,Telly.Hoeger@billy.biz,"{'street': 'Rex Trail', 'suite': 'Suite 280', ...",210.067.6132,elvis.io,"{'name': 'Johns Group', 'catchPhrase': 'Config..."
7,8,Nicholas Runolfsdottir V,Maxime_Nienow,Sherwood@rosamond.me,"{'street': 'Ellsworth Summit', 'suite': 'Suite...",586.493.6943 x140,jacynthe.com,"{'name': 'Abernathy Group', 'catchPhrase': 'Im..."
8,9,Glenna Reichert,Delphine,Chaim_McDermott@dana.io,"{'street': 'Dayna Park', 'suite': 'Suite 449',...",(775)976-6794 x41206,conrad.com,"{'name': 'Yost and Sons', 'catchPhrase': 'Swit..."
9,10,Clementina DuBuque,Moriah.Stanton,Rey.Padberg@karina.biz,"{'street': 'Kattie Turnpike', 'suite': 'Suite ...",024-648-3804,ambrose.net,"{'name': 'Hoeger LLC', 'catchPhrase': 'Central..."


In [74]:
#normalize address column
df_address = pd.json_normalize(users_df['address'])

#union of the two dataframes
users_df_test = pd.concat([users_df, df_address], axis=1)

users_df_test
df_resultado = users_df_test.drop('address', axis=1)

df_resultado

,id,name,username,email,phone,website,company,street,suite,city,zipcode,geo.lat,geo.lng
0,1,Leanne Graham,Bret,Sincere@april.biz,1-770-736-8031 x56442,hildegard.org,"{'name': 'Romaguera-Crona', 'catchPhrase': 'Mu...",Kulas Light,Apt. 556,Gwenborough,92998-3874,-37.3159,81.1496
1,2,Ervin Howell,Antonette,Shanna@melissa.tv,010-692-6593 x09125,anastasia.net,"{'name': 'Deckow-Crist', 'catchPhrase': 'Proac...",Victor Plains,Suite 879,Wisokyburgh,90566-7771,-43.9509,-34.4618
2,3,Clementine Bauch,Samantha,Nathan@yesenia.net,1-463-123-4447,ramiro.info,"{'name': 'Romaguera-Jacobson', 'catchPhrase': ...",Douglas Extension,Suite 847,McKenziehaven,59590-4157,-68.6102,-47.0653
3,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,493-170-9623 x156,kale.biz,"{'name': 'Robel-Corkery', 'catchPhrase': 'Mult...",Hoeger Mall,Apt. 692,South Elvis,53919-4257,29.4572,-164.2990
4,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,(254)954-1289,demarco.info,"{'name': 'Keebler LLC', 'catchPhrase': 'User-c...",Skiles Walks,Suite 351,Roscoeview,33263,-31.8129,62.5342
5,6,Mrs. Dennis Schulist,Leopoldo_Corkery,Karley_Dach@jasper.info,1-477-935-8478 x6430,ola.org,"{'name': 'Considine-Lockman', 'catchPhrase': '...",Norberto Crossing,Apt. 950,South Christy,23505-1337,-71.4197,71.7478
6,7,Kurtis Weissnat,Elwyn.Skiles,Telly.Hoeger@billy.biz,210.067.6132,elvis.io,"{'name': 'Johns Group', 'catchPhrase': 'Config...",Rex Trail,Suite 280,Howemouth,58804-1099,24.8918,21.8984
7,8,Nicholas Runolfsdottir V,Maxime_Nienow,Sherwood@rosamond.me,586.493.6943 x140,jacynthe.com,"{'name': 'Abernathy Group', 'catchPhrase': 'Im...",Ellsworth Summit,Suite 729,Aliyaview,45169,-14.3990,-120.7677
8,9,Glenna Reichert,Delphine,Chaim_McDermott@dana.io,(775)976-6794 x41206,conrad.com,"{'name': 'Yost and Sons', 'catchPhrase': 'Swit...",Dayna Park,Suite 449,Bartholomebury,76495-3109,24.6463,-168.8889
9,10,Clementina DuBuque,Moriah.Stanton,Rey.Padberg@karina.biz,024-648-3804,ambrose.net,"{'name': 'Hoeger LLC', 'catchPhrase': 'Central...",Kattie Turnpike,Suite 198,Lebsackbury,31428-2261,-38.2386,57.2232


In [5]:
users_test = pd.read_csv('users_extended.csv')
users_test


,USER_ID,USER_NAME,USER_USERNAME,USER_EMAIL,USER_PHONE,WEBSITE,USER_STREET,USER_SUITE,USER_CITY,USER_ZIPCODE,USER_LAT,USER_LNG
0,1,Leanne Graham,Bret,Sincere@april.biz,1-770-736-8031 x56442,hildegard.org,Kulas Light,Apt. 556,Gwenborough,92998-3874,-37.315900,81.149600
1,2,Ervin Howell,Antonette,Shanna@melissa.tv,010-692-6593 x09125,anastasia.net,Victor Plains,Suite 879,Wisokyburgh,90566-7771,-43.950900,-34.461800
2,3,Clementine Bauch,Samantha,Nathan@yesenia.net,1-463-123-4447,ramiro.info,Douglas Extension,Suite 847,McKenziehaven,59590-4157,-68.610200,-47.065300
3,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,493-170-9623 x156,kale.biz,Hoeger Mall,Apt. 692,South Elvis,53919-4257,29.457200,-164.299000
4,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,(254)954-1289,demarco.info,Skiles Walks,Suite 351,Roscoeview,33263,-31.812900,62.534200
...,...,...,...,...,...,...,...,...,...,...,...,...
55005,55006,Alexander Hughes,hamiltonthomas,martinezvictoria@example.net,846.209.1759x957,https://www.campbell.com/,525 Jackson Port,Apt. 878,North Stephanieton,19518,39.375597,97.178865
55006,55007,Paul Kim,robertrussell,staffordwilliam@example.net,325.916.7235x093,http://www.townsend.info/,526 Dennis Harbor Apt. 889,Apt. 691,New Marystad,01913,-22.499807,34.631837
55007,55008,Elizabeth Brewer,mercadolisa,charles49@example.org,001-804-520-9188x5650,http://brown.com/,149 Figueroa Walks Apt. 711,Suite 269,Lake Amandaland,57758,-25.364915,-140.349789
55008,55009,Amanda Underwood,wsimmons,jennifer14@example.org,625-816-9453,https://dean-jones.com/,8959 Karen Glen,Suite 848,West Brent,45819,-5.524719,-176.450230


In [533]:
users_test

,BOOKING_CODE,id,name,username,email,address,phone,website,company,company_id,test2_id
0,41DUBNM48VSCNEGTN,1,Leanne Graham,Bret,Sincere@april.biz,"{'street': 'Kulas Light', 'suite': 'Apt. 556',...",1-770-736-8031 x56442,hildegard.org,NaN,6,NaN
1,0KYFCF6T2XU1M8Z5C,2,Ervin Howell,Antonette,Shanna@melissa.tv,"{'street': 'Victor Plains', 'suite': 'Suite 87...",010-692-6593 x09125,anastasia.net,NaN,2,NaN
2,JHYSAP889W8EAPXDY,3,Clementine Bauch,Samantha,Nathan@yesenia.net,"{'street': 'Douglas Extension', 'suite': 'Suit...",1-463-123-4447,ramiro.info,NaN,25,NaN
3,RABKL9SU6VGST7BAT,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,"{'street': 'Hoeger Mall', 'suite': 'Apt. 692',...",493-170-9623 x156,kale.biz,"{'name': 'Robel-Corkery', 'catchPhrase': 'Mult...",29,8.0
4,WCRR8BTT4MV6B414X,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,"{'street': 'Skiles Walks', 'suite': 'Suite 351...",(254)954-1289,demarco.info,"{'name': 'Keebler LLC', 'catchPhrase': 'User-c...",44,7.0
5,UZLZ7H0S3JDYY6G2K,6,Mrs. Dennis Schulist,Leopoldo_Corkery,Karley_Dach@jasper.info,"{'street': 'Norberto Crossing', 'suite': 'Apt....",1-477-935-8478 x6430,ola.org,"{'name': 'Considine-Lockman', 'catchPhrase': '...",30,6.0
6,ENMEHSEF9UHKMH647,7,Kurtis Weissnat,Elwyn.Skiles,Telly.Hoeger@billy.biz,"{'street': 'Rex Trail', 'suite': 'Suite 280', ...",210.067.6132,elvis.io,"{'name': 'Johns Group', 'catchPhrase': 'Config...",41,5.0
7,LXK7S18X4FYDM9ZNY,8,Nicholas Runolfsdottir V,Maxime_Nienow,Sherwood@rosamond.me,"{'street': 'Ellsworth Summit', 'suite': 'Suite...",586.493.6943 x140,jacynthe.com,"{'name': 'Abernathy Group', 'catchPhrase': 'Im...",39,5.0
8,7MKF9EX554VSLSS5U,9,Glenna Reichert,Delphine,Chaim_McDermott@dana.io,"{'street': 'Dayna Park', 'suite': 'Suite 449',...",(775)976-6794 x41206,conrad.com,"{'name': 'Yost and Sons', 'catchPhrase': 'Swit...",42,8.0
9,BFYUDFNGXWUXM2567,10,Clementina DuBuque,Moriah.Stanton,Rey.Padberg@karina.biz,"{'street': 'Kattie Turnpike', 'suite': 'Suite ...",024-648-3804,ambrose.net,NaN,13,NaN


In [25]:


# Program to get timezone a phone number 
  
import phonenumbers 
from phonenumbers import timezone 
  
# Parsing String to Phone number 
phoneNumber = phonenumbers.parse("+919876543210") 
print(len(str(phoneNumber.national_number)))
  
# Pass the parsed phone number in below function 
timeZone = timezone.time_zones_for_number(phoneNumber) 
  
# It print the timezone of a phonenumber 
print(timeZone)


10
('Asia/Calcutta',)


In [26]:
def format_phone_number(phone_number):
    try:
        # Parse el número de teléfono utilizando la biblioteca phonenumbers
        parsed_number = phonenumbers.parse(phone_number, None)

        # Obtenga el formato nacional del número de teléfono
        national_format = phonenumbers.format_number(parsed_number, phonenumbers.PhoneNumberFormat.NATIONAL)

        # Compruebe las condiciones según las instrucciones proporcionadas
        if len(str(parsed_number.national_number)) == 9:
            # Números de teléfono de 9 dígitos: guardar tal cual
            return phone_number
        elif len(str(parsed_number.national_number)) == 10:
            if str(parsed_number.national_number)[0] == '1':
                # Números de teléfono de 10 dígitos que comienzan con 1: formato con paréntesis
                return national_format
            else:
                # Números de teléfono de 10 dígitos que no comienzan con 1: formato con paréntesis
                return national_format
        elif len(str(parsed_number.national_number)) == 11:
            if str(parsed_number.national_number)[0] == '1':
                # Números de teléfono de 11 dígitos que comienzan con 1: quitar el primer dígito y formato con paréntesis
                return national_format[1:]
            else:
                # Números de teléfono de 11 dígitos que no comienzan con 1: guardar tal cual
                return phone_number
        else:
            # Otros casos: guardar tal cual
            return phone_number
    except phonenumbers.NumberParseException:
        # Si no se puede analizar el número de teléfono, guárdelo tal cual
        return phone_number

In [27]:
users_test['USER_PHONE_FORMATTED'] = users_test['USER_PHONE'].apply(format_phone_number)

# Imprimir el DataFrame resultante
print(users_test)

       USER_ID         USER_NAME   USER_USERNAME  \
0            1     Leanne Graham            Bret   
1            2      Ervin Howell       Antonette   
2            3  Clementine Bauch        Samantha   
3            4  Patricia Lebsack        Karianne   
4            5  Chelsey Dietrich          Kamren   
...        ...               ...             ...   
55005    55006  Alexander Hughes  hamiltonthomas   
55006    55007          Paul Kim   robertrussell   
55007    55008  Elizabeth Brewer     mercadolisa   
55008    55009  Amanda Underwood        wsimmons   
55009    55010      Erica Newman         angel66   

                         USER_EMAIL             USER_PHONE  \
0                 Sincere@april.biz  1-770-736-8031 x56442   
1                 Shanna@melissa.tv    010-692-6593 x09125   
2                Nathan@yesenia.net         1-463-123-4447   
3         Julianne.OConner@kory.org      493-170-9623 x156   
4          Lucio_Hettinger@annie.ca          (254)954-1289   
...

In [28]:
users_test

,USER_ID,USER_NAME,USER_USERNAME,USER_EMAIL,USER_PHONE,WEBSITE,USER_STREET,USER_SUITE,USER_CITY,USER_ZIPCODE,USER_LAT,USER_LNG,USER_PHONE_FORMATTED
0,1,Leanne Graham,Bret,Sincere@april.biz,1-770-736-8031 x56442,hildegard.org,Kulas Light,Apt. 556,Gwenborough,92998-3874,-37.315900,81.149600,1-770-736-8031 x56442
1,2,Ervin Howell,Antonette,Shanna@melissa.tv,010-692-6593 x09125,anastasia.net,Victor Plains,Suite 879,Wisokyburgh,90566-7771,-43.950900,-34.461800,010-692-6593 x09125
2,3,Clementine Bauch,Samantha,Nathan@yesenia.net,1-463-123-4447,ramiro.info,Douglas Extension,Suite 847,McKenziehaven,59590-4157,-68.610200,-47.065300,1-463-123-4447
3,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,493-170-9623 x156,kale.biz,Hoeger Mall,Apt. 692,South Elvis,53919-4257,29.457200,-164.299000,493-170-9623 x156
4,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,(254)954-1289,demarco.info,Skiles Walks,Suite 351,Roscoeview,33263,-31.812900,62.534200,(254)954-1289
...,...,...,...,...,...,...,...,...,...,...,...,...,...
55005,55006,Alexander Hughes,hamiltonthomas,martinezvictoria@example.net,846.209.1759x957,https://www.campbell.com/,525 Jackson Port,Apt. 878,North Stephanieton,19518,39.375597,97.178865,846.209.1759x957
55006,55007,Paul Kim,robertrussell,staffordwilliam@example.net,325.916.7235x093,http://www.townsend.info/,526 Dennis Harbor Apt. 889,Apt. 691,New Marystad,01913,-22.499807,34.631837,325.916.7235x093
55007,55008,Elizabeth Brewer,mercadolisa,charles49@example.org,001-804-520-9188x5650,http://brown.com/,149 Figueroa Walks Apt. 711,Suite 269,Lake Amandaland,57758,-25.364915,-140.349789,001-804-520-9188x5650
55008,55009,Amanda Underwood,wsimmons,jennifer14@example.org,625-816-9453,https://dean-jones.com/,8959 Karen Glen,Suite 848,West Brent,45819,-5.524719,-176.450230,625-816-9453


In [36]:
import pandas as pd
from faker import Faker

# Supongamos que ya tienes un DataFrame llamado df_companies con las columnas mencionadas

# Crear un ejemplo de DataFrame
data_companies = {
    'USER_ID': range(1, 11),
    'COMPANY_NAME': ['Romaguera-Crona', 'Deckow-Crist', 'Romaguera-Jacobson', 'Robel-Corkery', 'Keebler LLC', 'Considine-Lockman', 'Johns Group', 'Abernathy Group', 'Yost and Sons', 'Hoeger LLC'],
    'COMPANY_CATCHPHRASE': ['Multi-layered client-server neural-net', 'Proactive didactic contingency', 'Face to face bifurcated interface', 'Multi-tiered zero tolerance productivity', 'User-centric fault-tolerant solution', 'Synchronised bottom-line interface', 'Configurable multimedia task-force', 'Implemented secondary concept', 'Switchable contextually-based project', 'Centralized empowering task-force'],
    'COMPANY_BS': ['harness real-time e-markets', 'synergize scalable supply-chains', 'e-enable strategic applications', 'transition cutting-edge web services', 'revolutionize end-to-end systems', 'e-enable innovative applications', 'generate enterprise e-tailers', 'e-enable extensible e-tailers', 'aggregate real-time technologies', 'target end-to-end models']
}

df_companies = pd.DataFrame(data_companies)

# Crear un objeto Faker
fake = Faker()

# Obtener 30,000 user_id únicos entre 1 y 30000
unique_user_ids = set()
while len(unique_user_ids) < 45:
    unique_user_ids.add(fake.random_int(min=11, max=60))

# Crear 30,000 registros adicionales con user_id únicos
for user_id in unique_user_ids:
    # Generar datos aleatorios usando Faker
    company_name = fake.company()
    company_catchphrase = fake.catch_phrase()
    company_bs = fake.bs()

    # Crear un nuevo registro
    new_record = {
        'USER_ID': user_id,
        'COMPANY_NAME': company_name,
        'COMPANY_CATCHPHRASE': company_catchphrase,
        'COMPANY_BS': company_bs
    }
    
    new_company_df = pd.DataFrame([new_record])

    df_companies = pd.concat([df_companies, new_company_df], ignore_index=True)


# Imprimir el DataFrame resultante
print(df_companies)


    USER_ID                    COMPANY_NAME  \
0         1                 Romaguera-Crona   
1         2                    Deckow-Crist   
2         3              Romaguera-Jacobson   
3         4                   Robel-Corkery   
4         5                     Keebler LLC   
5         6               Considine-Lockman   
6         7                     Johns Group   
7         8                 Abernathy Group   
8         9                   Yost and Sons   
9        10                      Hoeger LLC   
10       12                    Blake-Wright   
11       13                     Coleman Ltd   
12       14  Delacruz, Sweeney and Mcdonald   
13       15   Navarro, Clayton and Martinez   
14       16                      Fowler Ltd   
15       17                    Garrison Inc   
16       18                       Parks LLC   
17       19                Martinez-Freeman   
18       20       Anderson, Combs and Moran   
19       21                      Peters LLC   
20       22  

In [37]:
unique_user_ids = set()
while len(unique_user_ids) < 30:
    unique_user_ids.add(fake.random_int(min=1, max=50))
    
    
unique_user_ids

{1,
 3,
 4,
 5,
 6,
 8,
 10,
 11,
 15,
 16,
 17,
 18,
 20,
 22,
 24,
 25,
 27,
 29,
 30,
 31,
 35,
 36,
 38,
 39,
 41,
 44,
 45,
 46,
 47,
 48}

In [106]:
booking_df= pd.read_csv('raw_bookings.csv', sep=';')
booking_df

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,meal,country,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,agent,reservation_status,reservation_status_date
0,Resort Hotel,0,109,2016,January,1,0,1,2,0,BB,RUS,0,0,0,A,D,2.0,Check-Out,02/01/2016
1,Resort Hotel,0,109,2016,January,1,0,1,2,2,BB,RUS,0,0,0,H,H,2.0,Check-Out,02/01/2016
2,Resort Hotel,1,2,2016,January,1,0,1,2,0,BB,PRT,0,0,0,D,D,2.0,No-Show,01/01/2016
3,Resort Hotel,0,88,2016,January,1,0,2,2,0,HB,ARG,0,0,0,A,D,2.0,Check-Out,03/01/2016
4,Resort Hotel,1,20,2016,January,1,0,2,2,2,BB,PRT,0,0,0,G,G,2.0,Canceled,22/12/2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56702,City Hotel,0,17,2016,December,30,2,5,2,0,SC,FRA,0,0,0,A,A,9.0,Check-Out,06/01/2017
56703,City Hotel,0,177,2016,December,30,2,5,0,0,BB,PRT,0,0,0,E,K,NaN,Check-Out,06/01/2017
56704,City Hotel,0,107,2016,December,31,2,5,2,0,BB,FRA,0,0,0,A,B,2.0,Check-Out,07/01/2017
56705,City Hotel,0,310,2016,December,27,2,10,2,0,SC,DEU,0,0,0,A,A,9.0,Check-Out,08/01/2017


In [108]:
def bookings_extended(quantity_users, booking_df):
    # Crear un objeto Faker
    fake = Faker()
    
    df = pd.DataFrame()
    # Agregar columna 'USER_ID' con valores aleatorios entre 1 y 55010
    df['USER_ID'] = [fake.random_int(min=1, max=quantity_users) for i in range(len(df_reservas))]

    # Agregar columna 'LOCALIZADOR_RESERVA' con valores únicos
    booking_codes = set()
    
    while len(booking_codes) < len(booking_df):
        booking_code = fake.vin()
        booking_codes.add(booking_code)

    df['BOOKING_CODE'] = list(booking_codes)
    
    booking_df_extended = pd.concat([df,booking_df], axis=1)


    return booking_df_extended


In [429]:
bookings= pd.read_csv('raw_bookings.csv', sep=';')
bookings

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,meal,country,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,agent,reservation_status,reservation_status_date
0,Resort Hotel,0,109,2016,January,1,0,1,2,0,BB,RUS,0,0,0,A,D,2.0,Check-Out,02/01/2016
1,Resort Hotel,0,109,2016,January,1,0,1,2,2,BB,RUS,0,0,0,H,H,2.0,Check-Out,02/01/2016
2,Resort Hotel,1,2,2016,January,1,0,1,2,0,BB,PRT,0,0,0,D,D,2.0,No-Show,01/01/2016
3,Resort Hotel,0,88,2016,January,1,0,2,2,0,HB,ARG,0,0,0,A,D,2.0,Check-Out,03/01/2016
4,Resort Hotel,1,20,2016,January,1,0,2,2,2,BB,PRT,0,0,0,G,G,2.0,Canceled,22/12/2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56702,City Hotel,0,17,2016,December,30,2,5,2,0,SC,FRA,0,0,0,A,A,9.0,Check-Out,06/01/2017
56703,City Hotel,0,177,2016,December,30,2,5,0,0,BB,PRT,0,0,0,E,K,NaN,Check-Out,06/01/2017
56704,City Hotel,0,107,2016,December,31,2,5,2,0,BB,FRA,0,0,0,A,B,2.0,Check-Out,07/01/2017
56705,City Hotel,0,310,2016,December,27,2,10,2,0,SC,DEU,0,0,0,A,A,9.0,Check-Out,08/01/2017


In [450]:
bookings_c_a = bookings.copy()

In [453]:
bookings_c_a['arrival_date'] = pd.to_datetime(bookings_c_a['arrival_date_year'].astype(str) + '-' +
                                     bookings_c_a['arrival_date_month'].astype(str) + '-' +
                                     bookings_c_a['arrival_date_day_of_month'].astype(str), format='%Y-%B-%d')

In [454]:
bookings_c_a

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,country,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,agent,reservation_status,reservation_status_date,arrival_date
0,Resort Hotel,0,109,2016,January,1,0,1,2,0,...,RUS,0,0,0,A,D,2.0,Check-Out,02/01/2016,2016-01-01
1,Resort Hotel,0,109,2016,January,1,0,1,2,2,...,RUS,0,0,0,H,H,2.0,Check-Out,02/01/2016,2016-01-01
2,Resort Hotel,1,2,2016,January,1,0,1,2,0,...,PRT,0,0,0,D,D,2.0,No-Show,01/01/2016,2016-01-01
3,Resort Hotel,0,88,2016,January,1,0,2,2,0,...,ARG,0,0,0,A,D,2.0,Check-Out,03/01/2016,2016-01-01
4,Resort Hotel,1,20,2016,January,1,0,2,2,2,...,PRT,0,0,0,G,G,2.0,Canceled,22/12/2015,2016-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56702,City Hotel,0,17,2016,December,30,2,5,2,0,...,FRA,0,0,0,A,A,9.0,Check-Out,06/01/2017,2016-12-30
56703,City Hotel,0,177,2016,December,30,2,5,0,0,...,PRT,0,0,0,E,K,NaN,Check-Out,06/01/2017,2016-12-30
56704,City Hotel,0,107,2016,December,31,2,5,2,0,...,FRA,0,0,0,A,B,2.0,Check-Out,07/01/2017,2016-12-31
56705,City Hotel,0,310,2016,December,27,2,10,2,0,...,DEU,0,0,0,A,A,9.0,Check-Out,08/01/2017,2016-12-27


In [440]:
print(bookings_c_a['arrival_date'][0])
bookings_c_a['arrival_date'][0] + timedelta(days=35)

2016-01-01 00:00:00


Timestamp('2016-02-05 00:00:00')

In [514]:
def create_id(df, column_id, quantity=None):
    '''
    Creation of random ids in dataframe
    if a specified number of ids is required and the df size exceeds this number, 
    random values between 1 and this specified number will be created, otherwise unique id values will be created for each record.
    
    inputs:
    df: df to which ids are to be added
    column_id: name (str) to be assigned to the new ids column
    quantity(optional): number of ids to be added
    
    '''
    #create faker object
    fake = Faker()
    if quantity:
        df[column_id] = [fake.random_int(min=1, max=quantity) for i in range(len(df))]
    else:
        df[column_id] = np.random.permutation(np.arange(1, len(df) + 1))
    

In [515]:
create_id(bookings_c_a, 'hola_id', quantity=None)

In [516]:
bookings_c_a

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,agent,reservation_status,reservation_status_date,arrival_date,departure_date,reservation_date,hola_id
0,Resort Hotel,0,109,2016,January,1,0,1,2,0,...,0,A,D,2.0,Check-Out,02/01/2016,2016-01-01,2016-01-02,2015-09-14,50749
1,Resort Hotel,0,109,2016,January,1,0,1,2,2,...,0,H,H,2.0,Check-Out,02/01/2016,2016-01-01,2016-01-02,2015-09-14,49155
2,Resort Hotel,1,2,2016,January,1,0,1,2,0,...,0,D,D,2.0,No-Show,01/01/2016,2016-01-01,2016-01-02,2015-12-30,5992
3,Resort Hotel,0,88,2016,January,1,0,2,2,0,...,0,A,D,2.0,Check-Out,03/01/2016,2016-01-01,2016-01-03,2015-10-05,18800
4,Resort Hotel,1,20,2016,January,1,0,2,2,2,...,0,G,G,2.0,Canceled,22/12/2015,2016-01-01,2016-01-03,2015-12-12,48147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56702,City Hotel,0,17,2016,December,30,2,5,2,0,...,0,A,A,9.0,Check-Out,06/01/2017,2016-12-30,2017-01-06,2016-12-13,6205
56703,City Hotel,0,177,2016,December,30,2,5,0,0,...,0,E,K,NaN,Check-Out,06/01/2017,2016-12-30,2017-01-06,2016-07-06,1719
56704,City Hotel,0,107,2016,December,31,2,5,2,0,...,0,A,B,2.0,Check-Out,07/01/2017,2016-12-31,2017-01-07,2016-09-15,39350
56705,City Hotel,0,310,2016,December,27,2,10,2,0,...,0,A,A,9.0,Check-Out,08/01/2017,2016-12-27,2017-01-08,2016-02-21,24867


In [451]:
import pandas as pd

def get_departure_date(df):
    
    '''
    Function to calculate the departure date column.

    inputs:
    df = booking dataframe
    '''

    #calculate the column 'departure_date' by adding 'stays_in_week_end_nights' and 'stays_in_week_nights'.
    df['departure_date'] = df['arrival_date'] + pd.to_timedelta(df['stays_in_weekend_nights'] + df['stays_in_week_nights'], unit='D')


In [455]:
get_departure_date(bookings_c_a)

In [456]:
bookings_c_a

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,agent,reservation_status,reservation_status_date,arrival_date,departure_date
0,Resort Hotel,0,109,2016,January,1,0,1,2,0,...,0,0,0,A,D,2.0,Check-Out,02/01/2016,2016-01-01,2016-01-02
1,Resort Hotel,0,109,2016,January,1,0,1,2,2,...,0,0,0,H,H,2.0,Check-Out,02/01/2016,2016-01-01,2016-01-02
2,Resort Hotel,1,2,2016,January,1,0,1,2,0,...,0,0,0,D,D,2.0,No-Show,01/01/2016,2016-01-01,2016-01-02
3,Resort Hotel,0,88,2016,January,1,0,2,2,0,...,0,0,0,A,D,2.0,Check-Out,03/01/2016,2016-01-01,2016-01-03
4,Resort Hotel,1,20,2016,January,1,0,2,2,2,...,0,0,0,G,G,2.0,Canceled,22/12/2015,2016-01-01,2016-01-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56702,City Hotel,0,17,2016,December,30,2,5,2,0,...,0,0,0,A,A,9.0,Check-Out,06/01/2017,2016-12-30,2017-01-06
56703,City Hotel,0,177,2016,December,30,2,5,0,0,...,0,0,0,E,K,NaN,Check-Out,06/01/2017,2016-12-30,2017-01-06
56704,City Hotel,0,107,2016,December,31,2,5,2,0,...,0,0,0,A,B,2.0,Check-Out,07/01/2017,2016-12-31,2017-01-07
56705,City Hotel,0,310,2016,December,27,2,10,2,0,...,0,0,0,A,A,9.0,Check-Out,08/01/2017,2016-12-27,2017-01-08


In [457]:


def reservation_date(df):
    
    '''
    Function to calculate the reservation date column.

    inputs:
    df = bookings dataframe
    '''

    #calculate the reservation date column by removing lead time
    df['reservation_date'] = df['arrival_date'] - pd.to_timedelta(df['lead_time'], unit='D')

In [458]:
reservation_date(bookings_c_a)

bookings_c_a

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,agent,reservation_status,reservation_status_date,arrival_date,departure_date,reservation_date
0,Resort Hotel,0,109,2016,January,1,0,1,2,0,...,0,0,A,D,2.0,Check-Out,02/01/2016,2016-01-01,2016-01-02,2015-09-14
1,Resort Hotel,0,109,2016,January,1,0,1,2,2,...,0,0,H,H,2.0,Check-Out,02/01/2016,2016-01-01,2016-01-02,2015-09-14
2,Resort Hotel,1,2,2016,January,1,0,1,2,0,...,0,0,D,D,2.0,No-Show,01/01/2016,2016-01-01,2016-01-02,2015-12-30
3,Resort Hotel,0,88,2016,January,1,0,2,2,0,...,0,0,A,D,2.0,Check-Out,03/01/2016,2016-01-01,2016-01-03,2015-10-05
4,Resort Hotel,1,20,2016,January,1,0,2,2,2,...,0,0,G,G,2.0,Canceled,22/12/2015,2016-01-01,2016-01-03,2015-12-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56702,City Hotel,0,17,2016,December,30,2,5,2,0,...,0,0,A,A,9.0,Check-Out,06/01/2017,2016-12-30,2017-01-06,2016-12-13
56703,City Hotel,0,177,2016,December,30,2,5,0,0,...,0,0,E,K,NaN,Check-Out,06/01/2017,2016-12-30,2017-01-06,2016-07-06
56704,City Hotel,0,107,2016,December,31,2,5,2,0,...,0,0,A,B,2.0,Check-Out,07/01/2017,2016-12-31,2017-01-07,2016-09-15
56705,City Hotel,0,310,2016,December,27,2,10,2,0,...,0,0,A,A,9.0,Check-Out,08/01/2017,2016-12-27,2017-01-08,2016-02-21


In [283]:
import pandas as pd

# Crear un DataFrame de ejemplo
data = {'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9], 'D': [10, 11, 12]}
df = pd.DataFrame(data)

# Obtener el orden actual de las columnas
columnas_orden_actual = df.columns.tolist()

# Especificar el nuevo orden de las columnas
nuevo_orden = ['A', 'B', 'D', 'C']

# Reordenar el DataFrame con el nuevo orden de las columnas
df = df[nuevo_orden]

# Imprimir el DataFrame resultante
print(df)


   A  B   D  C
0  1  4  10  7
1  2  5  11  8
2  3  6  12  9


In [517]:
bookings_c_a

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,agent,reservation_status,reservation_status_date,arrival_date,departure_date,reservation_date,hola_id
0,Resort Hotel,0,109,2016,January,1,0,1,2,0,...,0,A,D,2.0,Check-Out,02/01/2016,2016-01-01,2016-01-02,2015-09-14,50749
1,Resort Hotel,0,109,2016,January,1,0,1,2,2,...,0,H,H,2.0,Check-Out,02/01/2016,2016-01-01,2016-01-02,2015-09-14,49155
2,Resort Hotel,1,2,2016,January,1,0,1,2,0,...,0,D,D,2.0,No-Show,01/01/2016,2016-01-01,2016-01-02,2015-12-30,5992
3,Resort Hotel,0,88,2016,January,1,0,2,2,0,...,0,A,D,2.0,Check-Out,03/01/2016,2016-01-01,2016-01-03,2015-10-05,18800
4,Resort Hotel,1,20,2016,January,1,0,2,2,2,...,0,G,G,2.0,Canceled,22/12/2015,2016-01-01,2016-01-03,2015-12-12,48147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56702,City Hotel,0,17,2016,December,30,2,5,2,0,...,0,A,A,9.0,Check-Out,06/01/2017,2016-12-30,2017-01-06,2016-12-13,6205
56703,City Hotel,0,177,2016,December,30,2,5,0,0,...,0,E,K,NaN,Check-Out,06/01/2017,2016-12-30,2017-01-06,2016-07-06,1719
56704,City Hotel,0,107,2016,December,31,2,5,2,0,...,0,A,B,2.0,Check-Out,07/01/2017,2016-12-31,2017-01-07,2016-09-15,39350
56705,City Hotel,0,310,2016,December,27,2,10,2,0,...,0,A,A,9.0,Check-Out,08/01/2017,2016-12-27,2017-01-08,2016-02-21,24867


In [518]:
def get_country_name(country_code):
    '''
    Use pycountry to identify code country in df
    '''
    try:
        country = pycountry.countries.get(alpha_3=country_code)
        if country:
            return country.name
        else:
            country = pycountry.countries.get(alpha_2=country_code)
            return country.name
    except (LookupError, AttributeError):
        return np.nan
    

In [522]:
get_country_name('VEN')

'Venezuela, Bolivarian Republic of'

In [528]:
def unique_country_list(df_column_w_codes):
    list_countries = list(df_column_w_codes.apply(lambda x: get_country_name(x)).unique())
    return list_countries
    

In [529]:
list_countries = unique_country_list(bookings_c_a.country)

In [530]:
list_countries

['Russian Federation',
 'Portugal',
 'Argentina',
 'France',
 'United Kingdom',
 'Germany',
 'Brazil',
 'Ireland',
 'United States',
 'Korea, Republic of',
 'Spain',
 'Denmark',
 'China',
 'Netherlands',
 'Hungary',
 nan,
 'Poland',
 'Belgium',
 'Luxembourg',
 'Austria',
 'Switzerland',
 'United Arab Emirates',
 'Italy',
 'Finland',
 'Tunisia',
 'Norway',
 'Romania',
 'Jamaica',
 'Albania',
 'Australia',
 'Sweden',
 'Croatia',
 'Hong Kong',
 'India',
 'Israel',
 'Czechia',
 'Iran, Islamic Republic of',
 'Georgia',
 'Algeria',
 'Morocco',
 'Andorra',
 'Turkey',
 'Mozambique',
 'South Africa',
 'Gibraltar',
 'Uruguay',
 'Belarus',
 'Estonia',
 'Lithuania',
 'Jersey',
 'Central African Republic',
 'Cyprus',
 'Greece',
 'Latvia',
 'Colombia',
 'Guernsey',
 'Kuwait',
 'Cuba',
 'Ukraine',
 'Cameroon',
 'Malaysia',
 'Thailand',
 'New Zealand',
 'Bosnia and Herzegovina',
 'Mauritius',
 'Nigeria',
 'Comoros',
 'Oman',
 'Suriname',
 'Mexico',
 'Chile',
 'Uganda',
 'Bulgaria',
 "Côte d'Ivoire",
 

In [532]:
def add_countries_to_df(df,df_column_w_codes):
    country_list = unique_country_list(df_column_w_codes)
    
    df['country'] = np.random.choice(country_list, len(df), replace=True)
    
    

In [534]:
bookings_c_a

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,agent,reservation_status,reservation_status_date,arrival_date,departure_date,reservation_date,hola_id
0,Resort Hotel,0,109,2016,January,1,0,1,2,0,...,0,A,D,2.0,Check-Out,02/01/2016,2016-01-01,2016-01-02,2015-09-14,50749
1,Resort Hotel,0,109,2016,January,1,0,1,2,2,...,0,H,H,2.0,Check-Out,02/01/2016,2016-01-01,2016-01-02,2015-09-14,49155
2,Resort Hotel,1,2,2016,January,1,0,1,2,0,...,0,D,D,2.0,No-Show,01/01/2016,2016-01-01,2016-01-02,2015-12-30,5992
3,Resort Hotel,0,88,2016,January,1,0,2,2,0,...,0,A,D,2.0,Check-Out,03/01/2016,2016-01-01,2016-01-03,2015-10-05,18800
4,Resort Hotel,1,20,2016,January,1,0,2,2,2,...,0,G,G,2.0,Canceled,22/12/2015,2016-01-01,2016-01-03,2015-12-12,48147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56702,City Hotel,0,17,2016,December,30,2,5,2,0,...,0,A,A,9.0,Check-Out,06/01/2017,2016-12-30,2017-01-06,2016-12-13,6205
56703,City Hotel,0,177,2016,December,30,2,5,0,0,...,0,E,K,NaN,Check-Out,06/01/2017,2016-12-30,2017-01-06,2016-07-06,1719
56704,City Hotel,0,107,2016,December,31,2,5,2,0,...,0,A,B,2.0,Check-Out,07/01/2017,2016-12-31,2017-01-07,2016-09-15,39350
56705,City Hotel,0,310,2016,December,27,2,10,2,0,...,0,A,A,9.0,Check-Out,08/01/2017,2016-12-27,2017-01-08,2016-02-21,24867


In [536]:
users_test

,BOOKING_CODE,id,name,username,email,address,phone,website,company,company_id,test2_id
0,41DUBNM48VSCNEGTN,1,Leanne Graham,Bret,Sincere@april.biz,"{'street': 'Kulas Light', 'suite': 'Apt. 556',...",1-770-736-8031 x56442,hildegard.org,NaN,6,NaN
1,0KYFCF6T2XU1M8Z5C,2,Ervin Howell,Antonette,Shanna@melissa.tv,"{'street': 'Victor Plains', 'suite': 'Suite 87...",010-692-6593 x09125,anastasia.net,NaN,2,NaN
2,JHYSAP889W8EAPXDY,3,Clementine Bauch,Samantha,Nathan@yesenia.net,"{'street': 'Douglas Extension', 'suite': 'Suit...",1-463-123-4447,ramiro.info,NaN,25,NaN
3,RABKL9SU6VGST7BAT,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,"{'street': 'Hoeger Mall', 'suite': 'Apt. 692',...",493-170-9623 x156,kale.biz,"{'name': 'Robel-Corkery', 'catchPhrase': 'Mult...",29,8.0
4,WCRR8BTT4MV6B414X,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,"{'street': 'Skiles Walks', 'suite': 'Suite 351...",(254)954-1289,demarco.info,"{'name': 'Keebler LLC', 'catchPhrase': 'User-c...",44,7.0
5,UZLZ7H0S3JDYY6G2K,6,Mrs. Dennis Schulist,Leopoldo_Corkery,Karley_Dach@jasper.info,"{'street': 'Norberto Crossing', 'suite': 'Apt....",1-477-935-8478 x6430,ola.org,"{'name': 'Considine-Lockman', 'catchPhrase': '...",30,6.0
6,ENMEHSEF9UHKMH647,7,Kurtis Weissnat,Elwyn.Skiles,Telly.Hoeger@billy.biz,"{'street': 'Rex Trail', 'suite': 'Suite 280', ...",210.067.6132,elvis.io,"{'name': 'Johns Group', 'catchPhrase': 'Config...",41,5.0
7,LXK7S18X4FYDM9ZNY,8,Nicholas Runolfsdottir V,Maxime_Nienow,Sherwood@rosamond.me,"{'street': 'Ellsworth Summit', 'suite': 'Suite...",586.493.6943 x140,jacynthe.com,"{'name': 'Abernathy Group', 'catchPhrase': 'Im...",39,5.0
8,7MKF9EX554VSLSS5U,9,Glenna Reichert,Delphine,Chaim_McDermott@dana.io,"{'street': 'Dayna Park', 'suite': 'Suite 449',...",(775)976-6794 x41206,conrad.com,"{'name': 'Yost and Sons', 'catchPhrase': 'Swit...",42,8.0
9,BFYUDFNGXWUXM2567,10,Clementina DuBuque,Moriah.Stanton,Rey.Padberg@karina.biz,"{'street': 'Kattie Turnpike', 'suite': 'Suite ...",024-648-3804,ambrose.net,NaN,13,NaN


In [537]:
add_countries_to_df(users_test, bookings_c_a.country)

In [538]:
users_test

,BOOKING_CODE,id,name,username,email,address,phone,website,company,company_id,test2_id,country
0,41DUBNM48VSCNEGTN,1,Leanne Graham,Bret,Sincere@april.biz,"{'street': 'Kulas Light', 'suite': 'Apt. 556',...",1-770-736-8031 x56442,hildegard.org,NaN,6,NaN,Honduras
1,0KYFCF6T2XU1M8Z5C,2,Ervin Howell,Antonette,Shanna@melissa.tv,"{'street': 'Victor Plains', 'suite': 'Suite 87...",010-692-6593 x09125,anastasia.net,NaN,2,NaN,India
2,JHYSAP889W8EAPXDY,3,Clementine Bauch,Samantha,Nathan@yesenia.net,"{'street': 'Douglas Extension', 'suite': 'Suit...",1-463-123-4447,ramiro.info,NaN,25,NaN,Malta
3,RABKL9SU6VGST7BAT,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,"{'street': 'Hoeger Mall', 'suite': 'Apt. 692',...",493-170-9623 x156,kale.biz,"{'name': 'Robel-Corkery', 'catchPhrase': 'Mult...",29,8.0,Netherlands
4,WCRR8BTT4MV6B414X,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,"{'street': 'Skiles Walks', 'suite': 'Suite 351...",(254)954-1289,demarco.info,"{'name': 'Keebler LLC', 'catchPhrase': 'User-c...",44,7.0,Ireland
5,UZLZ7H0S3JDYY6G2K,6,Mrs. Dennis Schulist,Leopoldo_Corkery,Karley_Dach@jasper.info,"{'street': 'Norberto Crossing', 'suite': 'Apt....",1-477-935-8478 x6430,ola.org,"{'name': 'Considine-Lockman', 'catchPhrase': '...",30,6.0,Monaco
6,ENMEHSEF9UHKMH647,7,Kurtis Weissnat,Elwyn.Skiles,Telly.Hoeger@billy.biz,"{'street': 'Rex Trail', 'suite': 'Suite 280', ...",210.067.6132,elvis.io,"{'name': 'Johns Group', 'catchPhrase': 'Config...",41,5.0,China
7,LXK7S18X4FYDM9ZNY,8,Nicholas Runolfsdottir V,Maxime_Nienow,Sherwood@rosamond.me,"{'street': 'Ellsworth Summit', 'suite': 'Suite...",586.493.6943 x140,jacynthe.com,"{'name': 'Abernathy Group', 'catchPhrase': 'Im...",39,5.0,"Bolivia, Plurinational State of"
8,7MKF9EX554VSLSS5U,9,Glenna Reichert,Delphine,Chaim_McDermott@dana.io,"{'street': 'Dayna Park', 'suite': 'Suite 449',...",(775)976-6794 x41206,conrad.com,"{'name': 'Yost and Sons', 'catchPhrase': 'Swit...",42,8.0,Burundi
9,BFYUDFNGXWUXM2567,10,Clementina DuBuque,Moriah.Stanton,Rey.Padberg@karina.biz,"{'street': 'Kattie Turnpike', 'suite': 'Suite ...",024-648-3804,ambrose.net,NaN,13,NaN,Cyprus


In [488]:
list_code_country

['RUS',
 'PRT',
 'ARG',
 'FRA',
 'GBR',
 'DEU',
 'BRA',
 'IRL',
 'USA',
 'KOR',
 'ESP',
 'DNK',
 'CHN',
 'NLD',
 'HUN',
 'POL',
 'CN',
 'BEL',
 'LUX',
 'AUT',
 'CHE',
 'ARE',
 'ITA',
 'FIN',
 'TUN',
 'NOR',
 'ROU',
 'JAM',
 'ALB',
 'AUS',
 'SWE',
 'HRV',
 'HKG',
 'IND',
 'ISR',
 'CZE',
 'IRN',
 'GEO',
 'DZA',
 'MAR',
 'AND',
 'TUR',
 'MOZ',
 'ZAF',
 'GIB',
 'URY',
 'BLR',
 'EST',
 'LTU',
 'JEY',
 'CAF',
 'CYP',
 'GRC',
 'LVA',
 'COL',
 'GGY',
 'KWT',
 'CUB',
 'UKR',
 'CMR',
 'MYS',
 'THA',
 'NZL',
 'BIH',
 'MUS',
 'NGA',
 'COM',
 'OMN',
 'SUR',
 'MEX',
 'CHL',
 'UGA',
 'BGR',
 'CIV',
 'SRB',
 'DOM',
 'JOR',
 'SYR',
 'PHL',
 'PRI',
 'JPN',
 'SGP',
 'ARM',
 'BDI',
 'AGO',
 'SAU',
 'LBN',
 'VNM',
 'AZE',
 'SVN',
 'PLW',
 'QAT',
 'EGY',
 'PER',
 'SVK',
 'CPV',
 'MDV',
 'MLT',
 'MWI',
 'ECU',
 'VEN',
 'IRQ',
 'SEN',
 'IDN',
 'TWN',
 'HND',
 'RWA',
 'PAK',
 'ZMB',
 'KHM',
 'MCO',
 'BGD',
 'ISL',
 'UZB',
 'KAZ',
 'IMN',
 'TJK',
 'NIC',
 'BEN',
 'MAC',
 'VGB',
 'CRI',
 'TZA',
 'GAB',
 'MKD',
 

In [475]:
def get_country_name(country_code):
    '''
    Use pycountry to identify code country in df
    '''
    try:
        country = pycountry.countries.get(alpha_3=country_code)
        if country:
            return country.name
        else:
            country = pycountry.countries.get(alpha_2=country_code)
            return country.name
    except (LookupError, AttributeError):
        return np.nan

In [485]:
countries = {
    'COUNTRY_ID' : list(list_code_country),
    'COUNTRY_NAME':[get_country_name(i) for i in list_code_country]
}

In [486]:
countries_df = pd.DataFrame(countries)

In [482]:
countries_df

,COUNTRY_ID,COUNTRY_NAME
0,RUS,Russian Federation
1,PRT,Portugal
2,ARG,Argentina
3,FRA,France
4,GBR,United Kingdom
...,...,...
147,LIE,Liechtenstein
148,GUY,Guyana
149,LCA,Saint Lucia
150,ATA,Antarctica


In [489]:
def create_dim_country_table():
    
    '''
    function to create the country_dim table
    '''
    
    #define the list with the code_country present in bookings_df
    list_code_country = ['RUS','PRT','ARG','FRA','GBR','DEU','BRA','IRL','USA','KOR','ESP','DNK','CHN','NLD','HUN',
 'POL','CN','BEL','LUX','AUT','CHE','ARE','ITA','FIN','TUN','NOR','ROU','JAM','ALB','AUS','SWE',
 'HRV','HKG','IND','ISR','CZE','IRN','GEO','DZA','MAR','AND','TUR','MOZ','ZAF','GIB','URY','BLR',
 'EST','LTU','JEY','CAF','CYP','GRC','LVA','COL','GGY','KWT','CUB','UKR','CMR','MYS','THA','NZL',
 'BIH','MUS','NGA','COM','OMN','SUR','MEX','CHL','UGA','BGR','CIV','SRB','DOM','JOR','SYR','PHL',
 'PRI','JPN','SGP','ARM','BDI','AGO','SAU','LBN','VNM','AZE','SVN','PLW','QAT','EGY','PER','SVK',
 'CPV','MDV','MLT','MWI','ECU','VEN','IRQ','SEN','IDN','TWN','HND','RWA','PAK','ZMB','KHM','MCO',
 'BGD','ISL','UZB','KAZ','IMN','TJK','NIC','BEN','MAC','VGB','CRI','TZA','GAB','MKD','GHA','TMP',
 'GLP','KEN','BFA','LKA','LBY','PAN','GNB','MLI','BHR','NAM','BOL','SYC','PRY','BRB','ABW','AIA',
 'SLV','DMA','PYF','LIE','GUY','LCA','ATA','MNE']
    
    #save in dict all countries
    countries = {
    'COUNTRY_ID' : list(list_code_country),
    'COUNTRY_NAME':[de.get_country_name(i) for i in list_code_country]
    }
    
    #convert in DataFrame
    dim_countries = pd.DataFrame(countries)
    
    return dim_countries

In [490]:
dim_countries = create_dim_country_table()

In [491]:
dim_countries

,COUNTRY_ID,COUNTRY_NAME
0,RUS,Russian Federation
1,PRT,Portugal
2,ARG,Argentina
3,FRA,France
4,GBR,United Kingdom
...,...,...
146,LIE,Liechtenstein
147,GUY,Guyana
148,LCA,Saint Lucia
149,ATA,Antarctica


In [492]:
list_code_meal = list(bookings.meal.dropna().unique())

In [493]:
list_code_meal

['BB', 'HB', 'Undefined', 'FB', 'SC']

In [494]:
def create_dim_country_table():
    
    '''
    function to create the meal_dim table
    '''
    
    #define the list with the meal_code present in bookings_df
    list_code_meal = ['BB', 'HB', 'FB', 'SC', 'Undefined']
    #define the list with the meal_name present in bookings_df
    list_name_meal = ['Bed and Breakfast', 'Half board', 'Full board', 'no meal package', 'no meal package']
    
    #save in dict all meals
    meals = {
    'MEAL_ID' : list_code_meal,
    'MEAL_NAME': list_name_meal
    }
    
    #convert in DataFrame
    dim_meals = pd.DataFrame(meals)
    
    return dim_meals


In [495]:
meals_dim = create_dim_country_table()

In [496]:
meals_dim

,MEAL_ID,MEAL_NAME
0,BB,Bed and Breakfast
1,HB,Half board
2,FB,Full board
3,SC,no meal package
4,Undefined,no meal package


In [497]:
list_code_hotel = list(bookings.hotel.dropna().unique())

In [498]:
list_code_hotel

['Resort Hotel', 'City Hotel']

In [499]:
def create_dim_hotel_table():


In [500]:
dim_hotels = create_dim_hotel_table()

In [501]:
dim_hotels

,HOTEL_ID,HOTEL_NAME
0,RH,Resort Hotel
1,CH,City Hotel


In [506]:
users_df
    
    
    


,id,name,username,email,address,phone,website,company
0,1,Leanne Graham,Bret,Sincere@april.biz,"{'street': 'Kulas Light', 'suite': 'Apt. 556',...",1-770-736-8031 x56442,hildegard.org,"{'name': 'Romaguera-Crona', 'catchPhrase': 'Mu..."
1,2,Ervin Howell,Antonette,Shanna@melissa.tv,"{'street': 'Victor Plains', 'suite': 'Suite 87...",010-692-6593 x09125,anastasia.net,"{'name': 'Deckow-Crist', 'catchPhrase': 'Proac..."
2,3,Clementine Bauch,Samantha,Nathan@yesenia.net,"{'street': 'Douglas Extension', 'suite': 'Suit...",1-463-123-4447,ramiro.info,"{'name': 'Romaguera-Jacobson', 'catchPhrase': ..."
3,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,"{'street': 'Hoeger Mall', 'suite': 'Apt. 692',...",493-170-9623 x156,kale.biz,"{'name': 'Robel-Corkery', 'catchPhrase': 'Mult..."
4,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,"{'street': 'Skiles Walks', 'suite': 'Suite 351...",(254)954-1289,demarco.info,"{'name': 'Keebler LLC', 'catchPhrase': 'User-c..."
5,6,Mrs. Dennis Schulist,Leopoldo_Corkery,Karley_Dach@jasper.info,"{'street': 'Norberto Crossing', 'suite': 'Apt....",1-477-935-8478 x6430,ola.org,"{'name': 'Considine-Lockman', 'catchPhrase': '..."
6,7,Kurtis Weissnat,Elwyn.Skiles,Telly.Hoeger@billy.biz,"{'street': 'Rex Trail', 'suite': 'Suite 280', ...",210.067.6132,elvis.io,"{'name': 'Johns Group', 'catchPhrase': 'Config..."
7,8,Nicholas Runolfsdottir V,Maxime_Nienow,Sherwood@rosamond.me,"{'street': 'Ellsworth Summit', 'suite': 'Suite...",586.493.6943 x140,jacynthe.com,"{'name': 'Abernathy Group', 'catchPhrase': 'Im..."
8,9,Glenna Reichert,Delphine,Chaim_McDermott@dana.io,"{'street': 'Dayna Park', 'suite': 'Suite 449',...",(775)976-6794 x41206,conrad.com,"{'name': 'Yost and Sons', 'catchPhrase': 'Swit..."
9,10,Clementina DuBuque,Moriah.Stanton,Rey.Padberg@karina.biz,"{'street': 'Kattie Turnpike', 'suite': 'Suite ...",024-648-3804,ambrose.net,"{'name': 'Hoeger LLC', 'catchPhrase': 'Central..."


In [508]:
def create_dim_companies_table(df_users, column_name):
    '''
    function to create a dim_companies table
    
    inputs:
    df_users = df with users raw data and extended. 
    '''
    #split users_df from the company column onward
    dim_companies = df_users.iloc[:,df_users.columns.get_loc('company'):]
    
    return dim_companies

In [509]:
create_dim_companies_table(users_df, )

,company
0,"{'name': 'Romaguera-Crona', 'catchPhrase': 'Mu..."
1,"{'name': 'Deckow-Crist', 'catchPhrase': 'Proac..."
2,"{'name': 'Romaguera-Jacobson', 'catchPhrase': ..."
3,"{'name': 'Robel-Corkery', 'catchPhrase': 'Mult..."
4,"{'name': 'Keebler LLC', 'catchPhrase': 'User-c..."
5,"{'name': 'Considine-Lockman', 'catchPhrase': '..."
6,"{'name': 'Johns Group', 'catchPhrase': 'Config..."
7,"{'name': 'Abernathy Group', 'catchPhrase': 'Im..."
8,"{'name': 'Yost and Sons', 'catchPhrase': 'Swit..."
9,"{'name': 'Hoeger LLC', 'catchPhrase': 'Central..."


In [512]:
def create_dim_users_table(df_users):
    '''
    function to create a dim_users table
    
    inputs:
    df_users = df with users raw data and extended. 
    '''
    #Split the users from the first column to the company column.
    dim_users = df_users.loc[:,:'website']
    
    return dim_users

In [513]:
create_dim_users_table(users_df)

,id,name,username,email,address,phone,website
0,1,Leanne Graham,Bret,Sincere@april.biz,"{'street': 'Kulas Light', 'suite': 'Apt. 556',...",1-770-736-8031 x56442,hildegard.org
1,2,Ervin Howell,Antonette,Shanna@melissa.tv,"{'street': 'Victor Plains', 'suite': 'Suite 87...",010-692-6593 x09125,anastasia.net
2,3,Clementine Bauch,Samantha,Nathan@yesenia.net,"{'street': 'Douglas Extension', 'suite': 'Suit...",1-463-123-4447,ramiro.info
3,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,"{'street': 'Hoeger Mall', 'suite': 'Apt. 692',...",493-170-9623 x156,kale.biz
4,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,"{'street': 'Skiles Walks', 'suite': 'Suite 351...",(254)954-1289,demarco.info
5,6,Mrs. Dennis Schulist,Leopoldo_Corkery,Karley_Dach@jasper.info,"{'street': 'Norberto Crossing', 'suite': 'Apt....",1-477-935-8478 x6430,ola.org
6,7,Kurtis Weissnat,Elwyn.Skiles,Telly.Hoeger@billy.biz,"{'street': 'Rex Trail', 'suite': 'Suite 280', ...",210.067.6132,elvis.io
7,8,Nicholas Runolfsdottir V,Maxime_Nienow,Sherwood@rosamond.me,"{'street': 'Ellsworth Summit', 'suite': 'Suite...",586.493.6943 x140,jacynthe.com
8,9,Glenna Reichert,Delphine,Chaim_McDermott@dana.io,"{'street': 'Dayna Park', 'suite': 'Suite 449',...",(775)976-6794 x41206,conrad.com
9,10,Clementina DuBuque,Moriah.Stanton,Rey.Padberg@karina.biz,"{'street': 'Kattie Turnpike', 'suite': 'Suite ...",024-648-3804,ambrose.net
